<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#NOT-FACTORIZED-CONDITION-(-1/1)" data-toc-modified-id="NOT-FACTORIZED-CONDITION-(-1/1)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NOT FACTORIZED CONDITION (-1/1)</a></span></li><li><span><a href="#FACTORIZED-CONDITION" data-toc-modified-id="FACTORIZED-CONDITION-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>FACTORIZED CONDITION</a></span></li></ul></div>

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import sem
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import ttest_rel, ttest_ind

In [100]:
import rpy2

In [101]:
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
# import R's "base" package
utils = importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
from rpy2.robjects.vectors import StrVector

packnames = ['lme4', 'optimx', 'pbkrtest', 'lmerTest',
             'ggplot2', 'dplyr', 'sjPlot', 'car']

# if there is no package then install it
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [102]:
%load_ext rpy2.ipython

from rpy2.robjects.packages import importr
# import R's "base" package
lme4 = importr('lme4')
optimx = importr('optimx')
pbkrtest = importr('pbkrtest')
lmerTest = importr('lmerTest')
ggplot = importr('ggplot2')
dplyr = importr('dplyr')
sjp = importr('sjPlot')
sjmisc = importr('sjmisc')
car = importr('car')
lattice = importr('lattice')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [103]:
home_dir = 'G:\My Drive\\researches\\nyu\\road-construction-local-dk\data_online_2022\\'
# home_dir = 'G:\.shortcut-targets-by-id\\1sYZNVBbYCrHcHjo-9xdKcA4wLnob9ZEq\\road_construction\\data\\2022_online\\R_analysis_data\\'
map_dir = 'active_map/'
data_dir  = 'data/preprocessed'
out_dir = home_dir + 'figures/figures_all/'
R_out_dir = home_dir + 'R_analysis_data/'

data_puzzle_level = pd.read_csv(R_out_dir +  'data.csv')
puzzleID_order_data = data_puzzle_level.sort_values(["subjects","puzzleID","condition"])
data_choice_level = pd.read_csv(R_out_dir +  'choice_level/choicelevel_data.csv')

single_condition_data = puzzleID_order_data[puzzleID_order_data['condition']==1].copy()
single_condition_data = single_condition_data.reset_index()

# NOT FACTORIZED CONDITION (-1/1)

In [104]:
condition = (np.array(data_puzzle_level.condition)-0.5)*2
subject = np.array(data_puzzle_level.subjects)
puzzleID = np.array(data_puzzle_level.puzzleID)
numUndo = np.array(data_puzzle_level.numUNDO)
mas = np.array(data_puzzle_level.mas)
num_loss = np.array(data_puzzle_level.mas) - np.array(data_puzzle_level.numCities)
numFullUndo = np.array(data_puzzle_level.numFullUndo)
# numFullUndo = np.sign(numFullUndo)

dat = pd.DataFrame({'subject':subject, 'condition':condition,'mas':mas,
                    'num_loss':num_loss,'puzzleID':puzzleID,'numUndo':numUndo,
                   'numFullUndo':numFullUndo, 'numFullUndo_factor':numFullUndo})



# error correction
dat.mas[dat[dat.num_loss == -1 ].index] = 9
num_loss = np.array(dat.mas) - np.array(data_puzzle_level.numCities)
dat.num_loss = num_loss

C:\ProgramData\Anaconda3\envs\base37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [105]:
%%R -i dat
# dat$numFullUndo <- factor(dat$numFullUndo)
# dat$numFullUndo_factor <- factor(dat$numFullUndo_factor)
dat$numFullUndo <- factor(dat$numFullUndo)
dat$subject <- factor(dat$subject)
# dat$condition <- factor(dat$condition)
dat$puzzleID <- factor(dat$puzzleID)

In [106]:
%%R
model6 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
summary(model6)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: 
num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition |  
    subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19026.5  19162.1  -9494.2  18988.5     9273 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9319 -0.6510 -0.3331  0.4763  8.5136 

Random effects:
 Groups   Name        Variance Std.Dev. Corr
 subject  (Intercept) 0.274949 0.52436      
          condition   0.004772 0.06908  0.60
 puzzleID (Intercept) 0.196142 0.44288      
Number of obs: 9292, groups:  subject, 101; puzzleID, 46

Fixed effects:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.46380    0.08516  -5.446 5.15e-08 ***
condition      -0.11495    0.01691  -6.799 1.06e-11 ***
numFullUndo1   -0.09099    0.05157  -1.764   0.0777 .  
numFullUndo2   -0.01723    0.08720  -0.198   0.8434    
numFullUndo3   -0.02962   

R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 0 (OK)
Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?



In [107]:
%%R
model4 = glmer(num_loss ~ condition + numFullUndo + (1+numFullUndo|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
summary(model4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) +  
    (1 + condition | subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19218.0  20095.9  -9486.0  18972.0     9169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9302 -0.6516 -0.3112  0.4658  7.7765 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                               
 subject  (Intercept)   0.275635 0.52501                                     
          condition     0.004625 0.06801  0.61                               
 puzzleID (Intercept)   0.198267 0.44527                                     
          numFullUndo1  0.007504 0.08663  -0.92                              
          numFullUndo2  0.084501 0.29069   0.60 -0.23                        
          numFullUndo3  0.143932 0.37938   0.97 -0.89  0.60                  


R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 4 (failure to converge in 10000 evaluations)
unable to evaluate scaled gradient
Model failed to converge: degenerate  Hessian with 4 negative eigenvalues
failure to converge in 10000 evaluations



In [108]:
%%R
# this case 
model0 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1|subject)
               , data=dat,
             family=poisson())
model1 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1+numFullUndo|subject)
               , data=dat,
             family=poisson())
model1_1 = glmer(num_loss ~ condition + numFullUndo + (1 +numFullUndo |puzzleID) + (1|subject)
               , data=dat,
             family=poisson())
model2 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
model2_1= glmer(num_loss ~ condition + numFullUndo + (1+condition|puzzleID) + (1|subject)
               , data=dat,
             family=poisson())
model3 = glmer(num_loss ~ condition + numFullUndo + (1+condition|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
summary(model4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) +  
    (1 + condition | subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19218.0  20095.9  -9486.0  18972.0     9169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9302 -0.6516 -0.3112  0.4658  7.7765 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                               
 subject  (Intercept)   0.275635 0.52501                                     
          condition     0.004625 0.06801  0.61                               
 puzzleID (Intercept)   0.198267 0.44527                                     
          numFullUndo1  0.007504 0.08663  -0.92                              
          numFullUndo2  0.084501 0.29069   0.60 -0.23                        
          numFullUndo3  0.143932 0.37938   0.97 -0.89  0.60                  


R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 4 (failure to converge in 10000 evaluations)
unable to evaluate scaled gradient
Model failed to converge: degenerate  Hessian with 4 negative eigenvalues
failure to converge in 10000 evaluations



In [109]:
%%R
anova(model0, model1)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model1: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + numFullUndo | subject)
       npar   AIC   BIC  logLik deviance  Chisq  Df Pr(>Chisq)
model0   17 19037 19159 -9501.7    19003                      
model1  121 19240 20103 -9498.9    18998 5.5584 104          1


In [110]:
%%R
anova(model0, model1_1)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model1_1: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) + (1 | subject)
         npar   AIC   BIC  logLik deviance  Chisq  Df Pr(>Chisq)
model0     17 19037 19159 -9501.7    19003                      
model1_1  121 19227 20090 -9492.3    18985 18.732 104          1


In [111]:
%%R
anova(model0, model2_1)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model2_1: num_loss ~ condition + numFullUndo + (1 + condition | puzzleID) + (1 | subject)
         npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)
model0     17 19037 19159 -9501.7    19003                     
model2_1   19 19037 19173 -9499.5    18999 4.4665  2     0.1072


In [112]:
%%R
anova(model0, model2)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model2: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition | subject)
       npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)    
model0   17 19037 19159 -9501.7    19003                         
model2   19 19027 19162 -9494.2    18989 14.879  2  0.0005876 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [113]:
%%R
anova(model2, model3)

Data: dat
Models:
model2: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition | subject)
model3: num_loss ~ condition + numFullUndo + (1 + condition | puzzleID) + (1 + condition | subject)
       npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)  
model2   19 19027 19162 -9494.2    18989                       
model3   21 19026 19176 -9491.9    18984 4.6272  2    0.09891 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [114]:
%%R
anova(model2, model4)

Data: dat
Models:
model2: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition | subject)
model4: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) + (1 + condition | subject)
       npar   AIC   BIC  logLik deviance  Chisq  Df Pr(>Chisq)
model2   19 19027 19162 -9494.2    18989                      
model4  123 19218 20096 -9486.0    18972 16.461 104          1


In [115]:
%%R
summary(model4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) +  
    (1 + condition | subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19218.0  20095.9  -9486.0  18972.0     9169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9302 -0.6516 -0.3112  0.4658  7.7765 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                               
 subject  (Intercept)   0.275635 0.52501                                     
          condition     0.004625 0.06801  0.61                               
 puzzleID (Intercept)   0.198267 0.44527                                     
          numFullUndo1  0.007504 0.08663  -0.92                              
          numFullUndo2  0.084501 0.29069   0.60 -0.23                        
          numFullUndo3  0.143932 0.37938   0.97 -0.89  0.60                  


R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 4 (failure to converge in 10000 evaluations)
unable to evaluate scaled gradient
Model failed to converge: degenerate  Hessian with 4 negative eigenvalues
failure to converge in 10000 evaluations



# FACTORIZED CONDITION

In [116]:
condition = (np.array(data_puzzle_level.condition))
subject = np.array(data_puzzle_level.subjects)
puzzleID = np.array(data_puzzle_level.puzzleID)
numUndo = np.array(data_puzzle_level.numUNDO)
mas = np.array(data_puzzle_level.mas)
num_loss = np.array(data_puzzle_level.mas) - np.array(data_puzzle_level.numCities)
numFullUndo = np.array(data_puzzle_level.numFullUndo)
# numFullUndo = np.sign(numFullUndo)

dat = pd.DataFrame({'subject':subject, 'condition':condition,'mas':mas,
                    'num_loss':num_loss,'puzzleID':puzzleID,'numUndo':numUndo,
                   'numFullUndo':numFullUndo, 'numFullUndo_factor':numFullUndo})



# error correction
dat.mas[dat[dat.num_loss == -1 ].index] = 9
num_loss = np.array(dat.mas) - np.array(data_puzzle_level.numCities)
dat.num_loss = num_loss

In [117]:
%%R -i dat
# dat$numFullUndo <- factor(dat$numFullUndo)
# dat$numFullUndo_factor <- factor(dat$numFullUndo_factor)
dat$numFullUndo <- factor(dat$numFullUndo)
dat$subject <- factor(dat$subject)
dat$condition <- factor(dat$condition)
dat$puzzleID <- factor(dat$puzzleID)

In [118]:
%%R
model6 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
summary(model6)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: 
num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition |  
    subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19026.5  19162.1  -9494.2  18988.5     9273 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9319 -0.6510 -0.3331  0.4762  8.5137 

Random effects:
 Groups   Name        Variance Std.Dev. Corr
 subject  (Intercept) 0.23629  0.4861       
          condition1  0.01908  0.1381   0.50
 puzzleID (Intercept) 0.19610  0.4428       
Number of obs: 9292, groups:  subject, 101; puzzleID, 46

Fixed effects:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.34884    0.08311  -4.197 2.70e-05 ***
condition1     -0.22989    0.03064  -7.503 6.25e-14 ***
numFullUndo1   -0.09102    0.04985  -1.826   0.0678 .  
numFullUndo2   -0.01718    0.08595  -0.200   0.8416    
numFullUndo3   -0.02961   

R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 0 (OK)
unable to evaluate scaled gradient
Model failed to converge: degenerate  Hessian with 1 negative eigenvalues



In [119]:
%%R
model4 = glmer(num_loss ~ condition + numFullUndo + (1+numFullUndo|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
summary(model4)

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) +  
    (1 + condition | subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19216.6  20094.4  -9485.3  18970.6     9169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9351 -0.6515 -0.3125  0.4674  7.7986 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                               
 subject  (Intercept)   0.235728 0.48552                                     
          condition1    0.018903 0.13749  0.51                               
 puzzleID (Intercept)   0.196351 0.44311                                     
          numFullUndo1  0.006723 0.08199  -1.00                              
          numFullUndo2  0.080076 0.28298   0.64 -0.64                        
          numFullUndo3  0.123540 0.35148   0.98 -0.98  0.55                  


R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 4 (failure to converge in 10000 evaluations)
boundary (singular) fit: see ?isSingular
failure to converge in 10000 evaluations



In [120]:
%%R
# this case 
model0 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1|subject)
               , data=dat,
             family=poisson())
model1 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1+numFullUndo|subject)
               , data=dat,
             family=poisson())
model1_1 = glmer(num_loss ~ condition + numFullUndo + (1 +numFullUndo |puzzleID) + (1|subject)
               , data=dat,
             family=poisson())
model2 = glmer(num_loss ~ condition + numFullUndo + (1|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
model2_1= glmer(num_loss ~ condition + numFullUndo + (1+condition|puzzleID) + (1|subject)
               , data=dat,
             family=poisson())
model3 = glmer(num_loss ~ condition + numFullUndo + (1+condition|puzzleID) + (1+condition|subject)
               , data=dat,
             family=poisson())
summary(model4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) +  
    (1 + condition | subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19216.6  20094.4  -9485.3  18970.6     9169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9351 -0.6515 -0.3125  0.4674  7.7986 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                               
 subject  (Intercept)   0.235728 0.48552                                     
          condition1    0.018903 0.13749  0.51                               
 puzzleID (Intercept)   0.196351 0.44311                                     
          numFullUndo1  0.006723 0.08199  -1.00                              
          numFullUndo2  0.080076 0.28298   0.64 -0.64                        
          numFullUndo3  0.123540 0.35148   0.98 -0.98  0.55                  


R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 4 (failure to converge in 10000 evaluations)
boundary (singular) fit: see ?isSingular
failure to converge in 10000 evaluations



In [121]:
%%R
anova(model0, model1)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model1: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + numFullUndo | subject)
       npar   AIC   BIC  logLik deviance  Chisq  Df Pr(>Chisq)
model0   17 19037 19159 -9501.7    19003                      
model1  121 19240 20103 -9498.9    18998 5.5659 104          1


In [122]:
%%R
anova(model0, model1_1)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model1_1: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) + (1 | subject)
         npar   AIC   BIC  logLik deviance  Chisq  Df Pr(>Chisq)
model0     17 19037 19159 -9501.7    19003                      
model1_1  121 19227 20090 -9492.3    18985 18.734 104          1


In [123]:
%%R
anova(model0, model2_1)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model2_1: num_loss ~ condition + numFullUndo + (1 + condition | puzzleID) + (1 | subject)
         npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)
model0     17 19037 19159 -9501.7    19003                     
model2_1   19 19037 19173 -9499.5    18999 4.4666  2     0.1072


In [124]:
%%R
anova(model0, model2)

Data: dat
Models:
model0: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 | subject)
model2: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition | subject)
       npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)    
model0   17 19037 19159 -9501.7    19003                         
model2   19 19027 19162 -9494.2    18989 14.879  2  0.0005876 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [125]:
%%R
anova(model2, model3)

Data: dat
Models:
model2: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition | subject)
model3: num_loss ~ condition + numFullUndo + (1 + condition | puzzleID) + (1 + condition | subject)
       npar   AIC   BIC  logLik deviance  Chisq Df Pr(>Chisq)  
model2   19 19027 19162 -9494.2    18989                       
model3   21 19026 19176 -9491.9    18984 4.6272  2     0.0989 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [126]:
%%R
anova(model2, model4)

Data: dat
Models:
model2: num_loss ~ condition + numFullUndo + (1 | puzzleID) + (1 + condition | subject)
model4: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) + (1 + condition | subject)
       npar   AIC   BIC  logLik deviance  Chisq  Df Pr(>Chisq)
model2   19 19027 19162 -9494.2    18989                      
model4  123 19217 20094 -9485.3    18971 17.925 104          1


In [127]:
%%R
summary(model4)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: num_loss ~ condition + numFullUndo + (1 + numFullUndo | puzzleID) +  
    (1 + condition | subject)
   Data: dat

     AIC      BIC   logLik deviance df.resid 
 19216.6  20094.4  -9485.3  18970.6     9169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9351 -0.6515 -0.3125  0.4674  7.7986 

Random effects:
 Groups   Name          Variance Std.Dev. Corr                               
 subject  (Intercept)   0.235728 0.48552                                     
          condition1    0.018903 0.13749  0.51                               
 puzzleID (Intercept)   0.196351 0.44311                                     
          numFullUndo1  0.006723 0.08199  -1.00                              
          numFullUndo2  0.080076 0.28298   0.64 -0.64                        
          numFullUndo3  0.123540 0.35148   0.98 -0.98  0.55                  


R[write to console]: 
Correlation matrix not shown by default, as p = 15 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




optimizer (Nelder_Mead) convergence code: 4 (failure to converge in 10000 evaluations)
boundary (singular) fit: see ?isSingular
failure to converge in 10000 evaluations

